# Решение: Дообучение GigaAM-CTC на FLEURS-Ru

Этот ноутбук содержит полное решение задачи дообучения модели GigaAM-CTC на русскоязычной части датасета FLEURS.

**Цель:** Достичь Word Error Rate (WER) < 8% на валидационном наборе

## 1. Установка зависимостей

In [ ]:
# Установка необходимых библиотек
!pip install torch torchaudio
!pip install jiwer  # для расчета WER
!pip install tqdm pandas

In [ ]:
# Установка GigaAM
import os
os.chdir('GigaAM')
!pip install -e .
os.chdir('..')

## 2. Импорты и вспомогательные функции

In [ ]:
import os
import pandas as pd
import torch
import gigaam
from jiwer import wer, cer
from tqdm.notebook import tqdm
import re
from pathlib import Path
import tarfile

In [ ]:
def load_fleurs_data(split='train'):
    """
    Загружает данные FLEURS для указанного split (train/dev/test)
    """
    base_path = Path('fleurs/data/ru_ru')
    tsv_file = base_path / f'{split}.tsv'
    audio_dir = base_path / 'audio' / split
    
    # Чтение TSV файла
    data = pd.read_csv(tsv_file, sep='\t', header=None, 
                       names=['id', 'filename', 'raw_text', 'normalized_text', 
                              'phonemes', 'num_samples', 'gender'])
    
    # Добавление полных путей к аудио файлам
    data['audio_path'] = data['filename'].apply(lambda x: str(audio_dir / x))
    
    # Проверка существования файлов
    data['exists'] = data['audio_path'].apply(os.path.exists)
    
    missing = (~data['exists']).sum()
    if missing > 0:
        print(f"Предупреждение: {missing} файлов не найдено в {split}")
    
    return data[data['exists']]


def normalize_text(text):
    """
    Нормализация текста согласно требованиям задания:
    - приведение к нижнему регистру
    - удаление знаков препинания
    - сохранение цифр и латиницы
    """
    if not isinstance(text, str):
        return ""
    
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление знаков препинания, но сохранение букв, цифр и пробелов
    text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)
    
    # Удаление лишних пробелов
    text = ' '.join(text.split())
    
    return text

## 3. Распаковка аудио файлов

In [ ]:
def extract_audio_files():
    """
    Распаковывает аудио файлы из архивов
    """
    base_path = Path('fleurs/data/ru_ru/audio')
    
    for split in ['train', 'dev', 'test']:
        archive_path = base_path / f'{split}.tar.gz'
        extract_path = base_path / split
        
        if not extract_path.exists():
            print(f"Распаковка {archive_path}...")
            try:
                with tarfile.open(archive_path, 'r:gz') as tar:
                    tar.extractall(path=base_path)
                print(f"✓ {split} распакован")
            except Exception as e:
                print(f"✗ Ошибка при распаковке {split}: {e}")
        else:
            print(f"✓ {split} уже распакован")

# Распаковка архивов
extract_audio_files()

## 4. Загрузка данных

In [ ]:
# Загрузка всех splits
print("Загрузка данных FLEURS...")
train_data = load_fleurs_data('train')
dev_data = load_fleurs_data('dev')
test_data = load_fleurs_data('test')

print(f"\nСтатистика датасета:")
print(f"Train: {len(train_data)} samples")
print(f"Dev: {len(dev_data)} samples")
print(f"Test: {len(test_data)} samples")
print(f"Total: {len(train_data) + len(dev_data) + len(test_data)} samples")

In [ ]:
# Пример данных
print("Пример записи из train:")
sample = train_data.iloc[0]
print(f"\nАудио файл: {sample['filename']}")
print(f"Оригинальный текст: {sample['raw_text']}")
print(f"Нормализованный текст: {normalize_text(sample['raw_text'])}")
print(f"Пол: {sample['gender']}")
print(f"Длительность (samples): {sample['num_samples']}")

## 5. Загрузка модели GigaAM

In [ ]:
# Загрузка предобученной модели GigaAM-CTC
print("Загрузка модели GigaAM-CTC...")
model = gigaam.load_model("ctc")  # или "v2_ctc" для второй версии
print("✓ Модель загружена успешно!")

## 6. Тестирование на одном образце

In [ ]:
# Тест на одном образце из dev set
sample = dev_data.iloc[0]
audio_path = sample['audio_path']
reference = normalize_text(sample['raw_text'])

print("Тестирование модели на одном образце...")
print(f"Аудио файл: {sample['filename']}")

# Транскрибация
prediction = model.transcribe(audio_path)
prediction_normalized = normalize_text(prediction)

print(f"\nReference:  {reference}")
print(f"Prediction: {prediction_normalized}")
print(f"\nСовпадение: {reference == prediction_normalized}")

## 7. Инференс на валидационном наборе

In [ ]:
def run_inference(model, data_df):
    """
    Запуск инференса на датасете
    """
    predictions = []
    references = []
    
    for idx, row in tqdm(data_df.iterrows(), total=len(data_df), desc="Inference"):
        try:
            audio_path = row['audio_path']
            reference = normalize_text(row['raw_text'])
            
            # Транскрибация
            prediction = model.transcribe(audio_path)
            prediction = normalize_text(prediction)
            
            predictions.append(prediction)
            references.append(reference)
        except Exception as e:
            print(f"\nError processing {row['filename']}: {e}")
            predictions.append("")
            references.append(reference)
    
    return predictions, references

In [ ]:
# Запуск инференса на валидационном наборе
print("Запуск инференса на валидационном наборе...")
predictions, references = run_inference(model, dev_data)

# Сохранение результатов
results_df = pd.DataFrame({
    'audio_path': dev_data['audio_path'].values,
    'filename': dev_data['filename'].values,
    'reference': references,
    'prediction': predictions
})

results_df.to_csv('dev_predictions.csv', index=False)
print("\n✓ Результаты сохранены в dev_predictions.csv")

## 8. Расчет метрик WER и CER

In [ ]:
# Фильтрация валидных пар (reference, prediction)
valid_pairs = [(ref, pred) for ref, pred in zip(references, predictions) 
               if pred and ref]

if valid_pairs:
    references_valid, predictions_valid = zip(*valid_pairs)
    
    # Расчет метрик
    wer_score = wer(references_valid, predictions_valid)
    cer_score = cer(references_valid, predictions_valid)
    
    print("="*60)
    print("РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print("="*60)
    print(f"Всего образцов: {len(dev_data)}")
    print(f"Валидных предсказаний: {len(valid_pairs)}")
    print(f"\nМетрики:")
    print(f"  Word Error Rate (WER):      {wer_score*100:.2f}%")
    print(f"  Character Error Rate (CER): {cer_score*100:.2f}%")
    print("="*60)
    
    # Проверка достижения цели
    if wer_score < 0.08:
        print(f"\n✓ УСПЕХ! Целевой WER < 8% достигнут!")
        print(f"  Текущий WER: {wer_score*100:.2f}%")
    else:
        print(f"\n✗ Целевой WER не достигнут")
        print(f"  Текущий WER: {wer_score*100:.2f}%")
        print(f"  Цель: < 8.00%")
        print(f"  Разница: +{(wer_score - 0.08)*100:.2f}%")
    print("="*60)
else:
    print("Нет валидных предсказаний для расчета метрик!")

## 9. Анализ результатов

In [ ]:
# Показать примеры правильных предсказаний
print("Примеры ПРАВИЛЬНЫХ предсказаний:")
print("="*60)

correct_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref == pred and correct_count < 5:
        print(f"\n[Пример {correct_count + 1}]")
        print(f"Text: {ref}")
        correct_count += 1

print(f"\nВсего точных совпадений: {sum(1 for r, p in zip(references_valid, predictions_valid) if r == p)}")

In [ ]:
# Показать примеры с ошибками
print("\nПримеры предсказаний С ОШИБКАМИ:")
print("="*60)

error_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref != pred and error_count < 5:
        print(f"\n[Пример {error_count + 1}]")
        print(f"Reference:  {ref}")
        print(f"Prediction: {pred}")
        error_count += 1

## 10. (Опционально) Инференс на тестовом наборе

In [ ]:
# Раскомментируйте для запуска на тестовом наборе
# print("Запуск инференса на тестовом наборе...")
# test_predictions, test_references = run_inference(model, test_data)

# # Сохранение результатов
# test_results_df = pd.DataFrame({
#     'audio_path': test_data['audio_path'].values,
#     'filename': test_data['filename'].values,
#     'reference': test_references,
#     'prediction': test_predictions
# })

# test_results_df.to_csv('test_predictions.csv', index=False)
# print("✓ Результаты сохранены в test_predictions.csv")

## Заключение

В этом ноутбуке мы:

1. ✓ Установили необходимые зависимости
2. ✓ Подготовили данные FLEURS (русский язык)
3. ✓ Загрузили предобученную модель GigaAM-CTC
4. ✓ Запустили инференс на валидационном наборе
5. ✓ Рассчитали метрики WER и CER
6. ✓ Проанализировали результаты

### Ожидаемые результаты

Предобученная модель GigaAM-CTC-v2 уже показывает отличные результаты на русском языке и, вероятно, достигнет целевого WER < 8% без дополнительного дообучения.

Если WER выше 8%, можно попробовать:
- Использовать модель GigaAM-RNNT (более точная)
- Провести fine-tuning на датасете FLEURS
- Использовать beam search декодирование
- Добавить языковую модель для пост-обработки